In [1]:
!nvidia-smi

Thu Jun  6 15:56:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 ...    Off | 00000000:03:00.0 Off |                  N/A |
| 27%   33C    P8               1W / 250W |      1MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# import os
# # Assign GPU number
# os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense,Dropout
import numpy as np

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

2024-06-06 15:56:14.270607: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
train = pd.read_csv('train.csv')

In [5]:
train['hindi'] = train['hindi'].astype(str)
train['english'] = train['english'].astype(str)

In [6]:
train['hindi'] = train['hindi'].str.strip() 
train['english'] = train['english'].str.strip().str.lower()

In [7]:
train = train[~train['english'].str.contains('\.')]
train = train[~train['english'].str.contains(',')]

In [8]:
train.head()

,Unnamed: 0,hindi,english
0,819069,अनाधिकृतपणे,anaadhikritapane
1,1160250,पॉडकास्टर,podcaster
2,810678,बेयंत,beyant
3,79522,बोझि,bojhee
4,1072591,बहानें,bahaanen


In [9]:
def isEnglish(s):
  try:
      s.encode(encoding='utf-8').decode('ascii')
  except UnicodeDecodeError:
      return False
  else:
      return True

train = train[train['english'].apply(isEnglish)]

In [10]:
sos = '^'
eos = '$'

train['english'] = sos + train['english'].astype(str) + eos
train.head()

,Unnamed: 0,hindi,english
0,819069,अनाधिकृतपणे,^anaadhikritapane$
1,1160250,पॉडकास्टर,^podcaster$
2,810678,बेयंत,^beyant$
3,79522,बोझि,^bojhee$
4,1072591,बहानें,^bahaanen$


In [11]:
hindi_text = train['hindi']
english_text = train['english']

In [12]:
print('hindi text shape :',hindi_text.shape)
print(hindi_text[0])
print('english text shape :',english_text.shape)
print(english_text[0])

hindi text shape : (100000,)
अनाधिकृतपणे
english text shape : (100000,)
^anaadhikritapane$


In [13]:
train_hindi_text, val_hindi_text, train_english_text, val_english_text = train_test_split(hindi_text, english_text, test_size=0.2, random_state=42)

In [14]:
max_input_len = max([len(text) for text in hindi_text])
print(max_input_len)
max_target_len = max([len(text) for text in english_text])
print(max_target_len)

26
29


In [15]:
input_chars = sorted(list(set(''.join(hindi_text))))
print(input_chars)
target_chars = sorted(list(set(''.join(english_text))))
print(target_chars)

['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ॊ', 'ो', 'ौ', '्']
['$', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [16]:
num_encoder_tokens = len(input_chars)
print(num_encoder_tokens)
num_decoder_tokens = len(target_chars)
print(num_encoder_tokens)

65
65


In [17]:
input_char_index = dict((char, i) for i, char in enumerate(input_chars))
print(input_char_index)
target_char_index = dict((char, i) for i, char in enumerate(target_chars))
print(target_char_index)

{'ँ': 0, 'ं': 1, 'ः': 2, 'अ': 3, 'आ': 4, 'इ': 5, 'ई': 6, 'उ': 7, 'ऊ': 8, 'ऋ': 9, 'ए': 10, 'ऐ': 11, 'ऑ': 12, 'ओ': 13, 'औ': 14, 'क': 15, 'ख': 16, 'ग': 17, 'घ': 18, 'ङ': 19, 'च': 20, 'छ': 21, 'ज': 22, 'झ': 23, 'ञ': 24, 'ट': 25, 'ठ': 26, 'ड': 27, 'ढ': 28, 'ण': 29, 'त': 30, 'थ': 31, 'द': 32, 'ध': 33, 'न': 34, 'प': 35, 'फ': 36, 'ब': 37, 'भ': 38, 'म': 39, 'य': 40, 'र': 41, 'ल': 42, 'ळ': 43, 'व': 44, 'श': 45, 'ष': 46, 'स': 47, 'ह': 48, '़': 49, 'ऽ': 50, 'ा': 51, 'ि': 52, 'ी': 53, 'ु': 54, 'ू': 55, 'ृ': 56, 'ॅ': 57, 'े': 58, 'ै': 59, 'ॉ': 60, 'ॊ': 61, 'ो': 62, 'ौ': 63, '्': 64}
{'$': 0, '^': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}


In [18]:
encoder_input_data = np.zeros((len(train_hindi_text), max_input_len, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(train_english_text), max_target_len, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(train_english_text), max_target_len, num_decoder_tokens), dtype='float32')

In [19]:
for i, (input_text, target_text) in enumerate(zip(train_hindi_text, train_english_text)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_char_index[char]] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_char_index[char]] = 1
        if t > 0:
            decoder_target_data[i, t - 1, target_char_index[char]] = 1

In [20]:
def custom_accuracy(y_true, y_pred):
    y_true_argmax = tf.argmax(y_true, axis=-1)
    y_pred_argmax = tf.argmax(y_pred, axis=-1)
    mask = tf.cast(tf.not_equal(y_true_argmax, 0), dtype=tf.float32)
    matches = tf.cast(tf.equal(y_true_argmax, y_pred_argmax), dtype=tf.float32)
    masked_matches = matches * mask
    accuracy = tf.reduce_sum(masked_matches) / tf.reduce_sum(mask)
    return accuracy

def custom_loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(tf.reduce_sum(y_true, axis=-1), 0), dtype=tf.float32)
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    masked_loss = loss * mask
    return tf.reduce_sum(masked_loss) / tf.reduce_sum(mask)


In [25]:
# model.save('my_custom2222_model.h5')
model = load_model('my_custom2222_model.h5', custom_objects={'custom_loss': custom_loss, 'custom_accuracy':custom_accuracy})


In [30]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True, dropout=0.3, recurrent_dropout=0.3)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True, dropout=0.3, recurrent_dropout=0.3)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=custom_loss,
              metrics=[custom_accuracy])

In [31]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 65)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 28)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 329728      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  291840      input_4[0][0]                    
                                                                 lstm_2[0][1]               

In [32]:
model.load_weights('my_custom2222_model.h5')

In [33]:
# history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=64,
#           epochs=50,
#           validation_split=0.2)

In [34]:
val_encoder_input_data = np.zeros((len(val_hindi_text), max_input_len, num_encoder_tokens), dtype='float32')
val_decoder_input_data = np.zeros((len(val_english_text), max_target_len, num_decoder_tokens), dtype='float32')
val_decoder_target_data = np.zeros((len(val_english_text), max_target_len, num_decoder_tokens), dtype='float32')

In [35]:
for i, (input_text, target_text) in enumerate(zip(val_hindi_text, val_english_text)):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t, input_char_index[char]] = 1
    for t, char in enumerate(target_text):
        val_decoder_input_data[i, t, target_char_index[char]] = 1
        if t > 0:
            val_decoder_target_data[i, t - 1, target_char_index[char]] = 1

In [36]:
# print(val_encoder_input_data)

In [39]:
scores = model.evaluate([val_encoder_input_data, val_decoder_input_data], val_decoder_target_data, verbose=1)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

625/625 [==============================] - 12s 19ms/step - loss: 0.2605 - custom_accuracy: 0.9162
Val loss: 0.2604626715183258
Val accuracy: 0.9161514639854431


In [40]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [41]:
import time
def beam_search_decode(input_seq, beam_width=2):
    start_time = time.time()  # Start measuring time
    states_value = encoder_model.predict(input_seq)
    
    start_token_index = target_char_index['^']
    end_token_index = target_char_index['$']
    sequences = [[list(), 0.0, states_value]]  # list of [sequence, score, states]
    
    for _ in range(max_target_len):
        all_candidates = []
        for seq, score, states in sequences:
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            if len(seq) > 0:
                last_token_index = seq[-1]
            else:
                last_token_index = start_token_index
            target_seq[0, 0, last_token_index] = 1.0
            
            output_tokens, h, c = decoder_model.predict([target_seq] + states)
            states_value = [h, c]
            
            top_k_indices = np.argsort(output_tokens[0, -1, :])[-beam_width:]
            for token_index in top_k_indices:
                candidate = [seq + [token_index], score - np.log(output_tokens[0, -1, token_index]), states_value]
                all_candidates.append(candidate)
        
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:beam_width]
    
    end_time = time.time()  
    latency = end_time - start_time  
    print(f'Latency for sequence: {latency:.4f} seconds')  
    
    # Convert all top sequences to decoded sentences and calculate their probabilities
    decoded_sentences = []
    for seq, score, _ in sequences:
        decoded_sentence = ''.join([target_chars[idx] for idx in seq])
        if decoded_sentence.endswith('$'):
            decoded_sentence = decoded_sentence[:-1]
        probability = np.exp(-score) # Convert negative log probability back to probability
        
        decoded_sentences.append((decoded_sentence, f'{probability:.4f}'))
    
    return decoded_sentences, latency 


In [42]:
def predict_output(input_text, beam_width=2):
    total_latency = 0
    word_count = 0
    results = []
    words = input_text.split(' ')
    for word in words:
        input_seq = np.zeros((1, max_input_len, num_encoder_tokens), dtype='float32')
        for t, char in enumerate(word):
            input_seq[0, t, input_char_index[char]] = 1
        decoded_sentences, latency = beam_search_decode(input_seq, beam_width)
        results.append(decoded_sentences)  # Collect all beam search results
        total_latency += latency
        word_count += 1
    average_latency = total_latency / word_count if word_count > 0 else 0
    print(f'Average latency: {average_latency:.4f} seconds')
    return results


In [43]:
text = 'विधायक'
print('Hindi text:',text)
print('Transliterated English text:',predict_output(text))

Hindi text: विधायक
Latency for sequence: 1.6390 seconds
Average latency: 1.6390 seconds
Transliterated English text: [[('vidhaayak$$$$$$$$$$$$$$$$$$$', '0.3942'), ('vidhayak$$$$$$$$$$$$$$$$$$$$', '0.1551')]]


In [44]:
text = 'जन्मदिवस'
print('Hindi text:',text)
print('Transliterated English text:',predict_output(text))

Hindi text: जन्मदिवस
Latency for sequence: 1.3949 seconds
Average latency: 1.3949 seconds
Transliterated English text: [[('janmadivas$$$$$$$$$$$$$$$$$$', '0.3458'), ('janmdivas$$$$$$$$$$$$$$$$$$$', '0.0679')]]


In [45]:
text = 'नालायक'
print('Hindi text:',text)
print('Transliterated English text:',predict_output(text))

Hindi text: नालायक
Latency for sequence: 1.4408 seconds
Average latency: 1.4408 seconds
Transliterated English text: [[('nalayak$$$$$$$$$$$$$$$$$$$$$', '0.3997'), ('naalayak$$$$$$$$$$$$$$$$$$$$', '0.0554')]]


In [46]:
text = 'विद्यापीठ'
print('Hindi text:',text)
print('Transliterated English text:',predict_output(text))

Hindi text: विद्यापीठ
Latency for sequence: 1.4117 seconds
Average latency: 1.4117 seconds
Transliterated English text: [[('vidyapith$$$$$$$$$$$$$$$$$$$', '0.3700'), ('vidyapeeth$$$$$$$$$$$$$$$$$$', '0.1949')]]


In [47]:
text = 'संकोच'
print('Hindi text:',text)
print('Transliterated English text:',predict_output(text))

Hindi text: संकोच
Latency for sequence: 1.4139 seconds
Average latency: 1.4139 seconds
Transliterated English text: [[('sankoch$$$$$$$$$$$$$$$$$$$$$', '0.4761'), ('sancoch$$$$$$$$$$$$$$$$$$$$$', '0.0458')]]


In [48]:
text = 'हनुमानाष्टक'
print('Hindi text:',text)
print('Transliterated English text:',predict_output(text))

Hindi text: हनुमानाष्टक
Latency for sequence: 1.4429 seconds
Average latency: 1.4429 seconds
Transliterated English text: [[('hanumanashtak$$$$$$$$$$$$$$$', '0.3469'), ('hanumaanashtak$$$$$$$$$$$$$$', '0.0669')]]
